임베딩 층(Embedding Layer)는 룩업(Lookup) 테이블이다
- 단어(text) -> 단어에 부여된 고유한 정수값(index) -> 임베딩 층 통과 -> 밀집 벡터(Dense Vector)
- 즉, Embedding Layer는 특정 정수(index)를 특정 밀집 벡터(dense vector)에 맵핑하는 역할이며, 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 밀집 벡터가 훈련된다

# 1. Embedding Layer

## Embedding Layer의 Lookup_Table 원리 이해하기

In [ ]:
train_data = 'you need to know how to code'

wordset = set(train_data.split()) # train_data로부터 중복 제거한 단어 집합을 생성

vocab = {word : i + 2 for i, word in enumerate(wordset)} # 2부터 시작해서 각 단어에 정수 인덱스를 부여
vocab['<unk>'] = 0 # 0은 unknown token
vocab['<pad>'] = 1 # 1은 padding token
print(vocab)

{'need': 2, 'how': 3, 'you': 4, 'to': 5, 'know': 6, 'code': 7, '<unk>': 0, '<pad>': 1}


In [ ]:
import torch
import torch.nn as nn

# 단어 집합의 크기만큼의 행을 가지는 테이블 생성.
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],  # <unk> : 0
                               [ 0.0,  0.0,  0.0],  # <pad> : 1
                               [ 0.2,  0.9,  0.3],  # need  : 2
                               [ 0.1,  0.5,  0.7],  # how   : 3
                               [ 0.2,  0.1,  0.8],  # you   : 4
                               [ 0.4,  0.1,  0.1],  # to    : 5
                               [ 0.1,  0.8,  0.9],  # know  : 6
                               [ 0.6,  0.1,  0.1]]) # code  : 7

In [ ]:
sample = 'you need to run'.split()
index = []

for word in sample:
  # 각 단어에 대해 Vocab의 정수 인덱스로 변환
  try:
    index.append(vocab[word])
  # Vocab에 없는 단어에 대해서는 <unk>으로
  except KeyError:
    index.append(vocab['<unk>'])

# embedding_table이 tensor이기 때문에 tensor 타입으로 변형
index = torch.LongTensor(index)
index

tensor([4, 2, 5, 0])

In [ ]:
# row(단어 개수)는 index(4, 2, 5, 0)를 참조하고 그 때마다 col(임베딩 벡터의 차원)은 all(:)
lookup_result = embedding_table[index, :]
print(lookup_result)

tensor([[0.2000, 0.1000, 0.8000],
        [0.2000, 0.9000, 0.3000],
        [0.4000, 0.1000, 0.1000],
        [0.0000, 0.0000, 0.0000]])


```
tensor([[0.2000, 0.1000, 0.8000],   # you
        [0.2000, 0.9000, 0.3000],   # need
        [0.4000, 0.1000, 0.1000],   # to
        [0.0000, 0.0000, 0.0000]])  # <unk>
```

## nn.Embedding()으로 임베딩 층(Embedding Layer) 사용하기

In [ ]:
train_data = 'you need to know how to code'

wordset = set(train_data.split()) # train_data로부터 중복 제거한 단어 집합을 생성

vocab = {word : i + 2 for i, word in enumerate(wordset)} # 2부터 시작해서 각 단어에 정수 인덱스를 부여
vocab['<unk>'] = 0 # 0은 unknown token
vocab['<pad>'] = 1 # 1은 padding token
print(vocab)

{'need': 2, 'how': 3, 'you': 4, 'to': 5, 'know': 6, 'code': 7, '<unk>': 0, '<pad>': 1}


In [ ]:
embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=3, padding_idx=1)
# num_embeddings : 임베딩할 단어의 개수
# embedding_dim : 임베딩할 벡터의 차원
# padding_idx : 패딩을 위한 토큰의 인덱스

In [ ]:
print(embedding_layer.weight)
# 앞에서는 임의로 생성했던 Lookup Table이 nn.Embedding으로 생성된 것을 확인

Parameter containing:
tensor([[-0.3912,  0.2304, -0.2282],
        [ 0.0000,  0.0000,  0.0000],
        [-1.2417,  0.6831,  1.2010],
        [-0.6814,  0.9481,  0.9033],
        [ 1.3358,  0.1982,  0.9304],
        [ 0.4873, -0.0161, -0.5158],
        [ 1.5266, -1.1556,  0.5876],
        [-2.0050, -1.1502, -0.2996]], requires_grad=True)


# 2. Pretrained Word Embedding

In [1]:
!pip install gensim

## Pretrained Word Embedding을 사용하지 않는 경우

In [2]:
import numpy as np
from collections import Counter
import gensim

# 7개의 문장과 긍/부정 판단을 위한 레이블 데이터 생성
sentences = ['nice great best amazing',
             'stop lies',
             'pitiful nerd',
             'excellent work',
             'supreme quality',
             'bad',
             'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [3]:
tokenized_sentences = [sent.split() for sent in sentences]
print('단어 토큰화 된 결과 :', tokenized_sentences)

단어 토큰화 된 결과 : [['nice', 'great', 'best', 'amazing'], ['stop', 'lies'], ['pitiful', 'nerd'], ['excellent', 'work'], ['supreme', 'quality'], ['bad'], ['highly', 'respectable']]


In [4]:
word_list = []
# 모든 문장의 토큰화된 단어를 word_list에 넣음
for sent in tokenized_sentences:
  for word in sent:
    word_list.append(word)

word_counts = Counter(word_list)
print('총 단어수 :', len(word_counts))

총 단어수 : 15


In [5]:
# 등장 빈도순으로 정렬
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab)

['nice', 'great', 'best', 'amazing', 'stop', 'lies', 'pitiful', 'nerd', 'excellent', 'work', 'supreme', 'quality', 'bad', 'highly', 'respectable']


In [6]:
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab):
  word_to_index[word] = index + 2

vocab_size = len(word_to_index)
print("패딩 토큰, UNK 토큰을 고려한 단어 집합의 크기 :", vocab_size) # 15 + 2(PAD, UNK)

패딩 토큰, UNK 토큰을 고려한 단어 집합의 크기 : 17


In [7]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent: # 각 문장의 단어에 대해서
      try:
        index_sequences.append(word_to_index[word]) # 사전 내 단어에 할당된 index를 index_sequences에 추가
      except KeyError: # OOV(사전에 없는 경우)
        index_sequences.append(word_to_index['<UNK>']) # UNK 토큰에 할당된 index를 index_sequences에 추가
    encoded_X_data.append(index_sequences)
  return encoded_X_data # 각 문장마다 index sequence로 변환한 것들의 모음(리스트)

X_encoded = texts_to_sequences(tokenized_sentences, word_to_index)
print(X_encoded)

[[2, 3, 4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14], [15, 16]]


In [8]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :', max_len)

최대 길이 : 4


In [9]:
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

X_train = pad_sequences(X_encoded, max_len)
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 2  3  4  5]
 [ 6  7  0  0]
 [ 8  9  0  0]
 [10 11  0  0]
 [12 13  0  0]
 [14  0  0  0]
 [15 16  0  0]]


In [45]:
print(y_train)

[1 0 0 1 1 0 1]


In [10]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [12]:
class SimpleModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(SimpleModel, self).__init__() # nn.Module의 __init__을 상속 받고
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(embedding_dim * max_len, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    embedded = self.embedding(x) # shape : (batch_size, sentence_length, embedding_dim)
    flattened = self.flatten(embedded) # shape : (batch_size, sentence_length * embedding_dim)
    output = self.fc(flattened) # shape : (batch_size, 1)
    return self.sigmoid(output)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_dim = 100
simple_model = SimpleModel(vocab_size, embedding_dim).to(device)

In [13]:
criterion = nn.BCELoss()
optimizer = Adam(simple_model.parameters())

In [15]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [16]:
print(len(train_dataloader)) # 7개의 데이터를 batch_size 2로 하면 총 4번의 iteration이 된다

4


In [17]:
for epoch in range(10):
  for inputs, targets in train_dataloader:
    inputs, targets = inputs.to(device), targets.to(device) # GPU에 해당 batch의 input과 target을 올리고

    optimizer.zero_grad()                   # 이전 epoch에서 계산된 기울기를 초기화
    outputs = simple_model(inputs).view(-1) # 모델의 예측 계산 & 1차원 텐서로 변환
    loss = criterion(outputs, targets)      # 손실값 계산
    loss.backward()                         # 손실값을 토대로 각 파라미터 업데이트에 필요한 기울기 계산

    optimizer.step()                        # 계산된 기울기를 바탕으로 파라미터 업데이트
  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 1.1613306999206543
Epoch 2, Loss: 0.8831257224082947
Epoch 3, Loss: 0.6458916068077087
Epoch 4, Loss: 0.47528305649757385
Epoch 5, Loss: 0.3627965748310089
Epoch 6, Loss: 0.29160618782043457
Epoch 7, Loss: 0.24681179225444794
Epoch 8, Loss: 0.21781857311725616
Epoch 9, Loss: 0.1976604014635086
Epoch 10, Loss: 0.18192654848098755


## Pretrained Word Embedding을 사용하는 경우

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j

In [29]:
# 사전 학습된 Word2Vec 모델 로드
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [33]:
embedding_matrix = np.zeros((vocab_size, 300)) # 사용하는 Word2Vec 모델이 300차원이기 때문에 300차원으로 설정, 다르면 오류 발생
print('임베딩 행렬의 크기 :', embedding_matrix.shape) # row가 어휘의 개수, col이 각 어휘별 임베딩 차원

임베딩 행렬의 크기 : (17, 300)


In [34]:
# 해당하는 단어가 pretrained Word2Vec에 있는 단어이면, 그 단어에 대한 사전 학습된 임베딩 벡터를 반환하도록
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [35]:
# <PAD>를 위한 0번과 <UNK>를 위한 1번은 실제 단어가 아니므로 맵핑에서 제외
for word, i in word_to_index.items():
    if i > 2: # 0, 1은 <PAD>, <UNK>이므로
      temp = get_vector(word)
      if temp is not None:
          embedding_matrix[i] = temp

In [36]:
# <PAD>나 <UNK>의 경우는 사전 훈련된 임베딩이 들어가지 않아서 0벡터임
print(embedding_matrix[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [37]:
word_to_index['great']

3

In [38]:
# word2vec_model에서 'great'의 임베딩 벡터
# embedding_matrix[3]이 일치하는지 체크
np.all(word2vec_model['great'] == embedding_matrix[3])

True

In [39]:
word2vec_model['great'].shape

(300,)

In [40]:
class PretrainedEmbeddingModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(PretrainedEmbeddingModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
    self.embedding.weight.requires_grad = True
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(embedding_dim*max_len, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    embedded = self.embedding(x)
    flattened = self.flatten(embedded)
    output = self.fc(flattened)
    return self.sigmoid(output)

In [41]:
pretrained_embedding_model = PretrainedEmbeddingModel(vocab_size, 300).to(device)

In [42]:
criterion = nn.BCELoss()
optimzer = Adam(pretrained_embedding_model.parameters())

In [43]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [44]:
for epoch in range(10):
  for inputs, targets in train_dataloader:
    inputs, targets = inputs.to(device), targets.to(device) # GPU에 해당 batch의 input과 target을 올리고

    optimizer.zero_grad()                   # 이전 epoch에서 계산된 기울기를 초기화
    outputs = simple_model(inputs).view(-1) # 모델의 예측 계산 & 1차원 텐서로 변환
    loss = criterion(outputs, targets)      # 손실값 계산
    loss.backward()                         # 손실값을 토대로 각 파라미터 업데이트에 필요한 기울기 계산

    optimizer.step()                        # 계산된 기울기를 바탕으로 파라미터 업데이트
  print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.168012335896492
Epoch 2, Loss: 0.15463373064994812
Epoch 3, Loss: 0.14142407476902008
Epoch 4, Loss: 0.1285470575094223
Epoch 5, Loss: 0.11636266857385635
Epoch 6, Loss: 0.1052008792757988
Epoch 7, Loss: 0.09525743871927261
Epoch 8, Loss: 0.08658034354448318
Epoch 9, Loss: 0.07910498231649399
Epoch 10, Loss: 0.07270035147666931


사전 학습된 모델을 사용해서 첫 Epoch부터 loss가 매우 낮은 상태이며 학습을 진행한 이후 사전학습 Embedding model을 사용하지 않은 것에 비해 loss가 더 낮아졌다